In [15]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import tqdm
import psycopg2
import sqlalchemy
import pandas as pd 

In [16]:
# conn = psycopg2.connect("host=localhost dbname=gradient_local user=postgres") 
# cursor = conn.cursor()

db_uri = "postgresql://postgres:@localhost/gradient_local"

get_items_query = "SELECT * FROM items;"
get_users_query = "SELECT * FROM users;"
get_fct_query = "SET constraint_exclusion = on; SELECT * FROM fct_hourly_metric;"

In [17]:
items_df = pd.read_sql(get_items_query, db_uri)
items_df.head()

,item_key,created_at,user_id,bucket_key,type
0,d6a9a727-3c39-41bc-a46a-1027a85a0fac,2023-07-14 14:35:37,24549,tiny-texts-jk9apq,txt
1,24153e9f-c757-448a-b854-ade8b23feff5,2023-07-14 14:35:25,32109,tiny-images-jk9apq,img
2,5921d0f8-8989-495d-bf28-247743063659,2023-07-14 14:33:48,33961,tiny-texts-jk9apq,txt
3,fdc57820-5a49-4467-b8ce-44e814948d0e,2023-07-14 14:31:47,2295,tiny-texts-jk9apq,txt
4,e14c4b33-b1e5-42a4-920d-b1c10a5a5545,2023-07-14 14:31:02,26137,tiny-images-jk9apq,img


In [18]:
users_df = pd.read_sql(get_users_query, db_uri)
users_df.head()

/var/folders/0y/d_717f6d3wv7j3stmqzgt9_c0000gp/T/ipykernel_65740/1831833833.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users_df = pd.read_sql(get_users_query, conn)


,id,gender,country,age
0,1,male,AlgoBay,40.0
1,2,female,GraphTown,19.0
2,3,female,BayesianBourg,7.0
3,4,male,AdaLove,21.0
4,36,male,AlgoBay,26.0


In [19]:
fct_df = pd.read_sql(get_fct_query, conn)
fct_df.head()

/var/folders/0y/d_717f6d3wv7j3stmqzgt9_c0000gp/T/ipykernel_65740/1641039730.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fct_df = pd.read_sql(get_fct_query, conn)


,date_stamp,time_stamp,evnt_stamp,user_id,item_id,evt_type,session_id
0,2023-07-25,2023-07-25 13:00:00,1690291720,24473.0,ab1419f7-6789-4d2a-ae0a-b6ffcc58adfd,view,29e43e40-44cf-4f79-9b0d-076f3cb66c43
1,2023-07-25,2023-07-25 13:00:00,1690291725,24473.0,1a0f6282-8874-432c-813a-a8daa3021e55,view,29e43e40-44cf-4f79-9b0d-076f3cb66c43
2,2023-07-25,2023-07-25 13:00:00,1690291725,24473.0,fd760756-961b-4e12-8d75-090902e74921,view,29e43e40-44cf-4f79-9b0d-076f3cb66c43
3,2023-07-25,2023-07-25 13:00:00,1690291725,24473.0,None,stop,29e43e40-44cf-4f79-9b0d-076f3cb66c43
4,2023-07-25,2023-07-25 13:00:00,1690291729,19301.0,f1b4453a-1048-46a2-9d54-bd37571a7e6f,view,c18e3c3e-a5c1-4f9a-b78e-f9a104ded275
